# Average Directional Index (ADX) - Complete Trading Tutorial

## Overview
This notebook provides a comprehensive tutorial on the Average Directional Index (ADX) indicator, including:
- Concept and theory
- Mathematical formula and calculation
- Manual implementation in Python
- TradingView integration
- Binance platform usage
- Practical trading strategies
- Backtesting examples

## 1. Introduction to Average Directional Index (ADX)

### What is ADX?
The Average Directional Index (ADX) is a technical analysis indicator used to determine the strength of a trend. The ADX does not indicate trend direction, only trend strength. It is composed of two other indicators: the Positive Directional Indicator (+DI) and the Negative Directional Indicator (-DI).

### Why Use ADX?
- **Trend Strength Measurement**: ADX quantifies the strength of a trend, helping traders avoid ranging markets.
- **Filter for Strategies**: It can be used as a filter for trend-following strategies, ensuring that a trader only enters when the trend is strong enough.
- **Trend Direction Confirmation**: While ADX itself is non-directional, the +DI and -DI lines can be used to determine trend direction.

### ADX Values
- **0-25**: Absent or weak trend
- **25-50**: Strong trend
- **50-75**: Very strong trend
- **75-100**: Extremely strong trend

## 2. Mathematical Formula

### ADX Formula
ADX calculation involves several steps:
1. **Calculate True Range (TR)**: `TR = max[(High - Low), abs(High - Previous_Close), abs(Low - Previous_Close)]`
2. **Calculate Directional Movement (+DM, -DM)**:
   - `+DM = Current_High - Previous_High`
   - `-DM = Previous_Low - Current_Low`
3. **Calculate Smoothed TR, +DM, -DM (usually over 14 periods)**
4. **Calculate Directional Indicators (+DI, -DI)**:
   - `+DI = 100 * (Smoothed_+DM / Smoothed_TR)`
   - `-DI = 100 * (Smoothed_-DM / Smoothed_TR)`
5. **Calculate Directional Index (DX)**: `DX = 100 * (abs(+DI - -DI) / (+DI + -DI))`
6. **Calculate ADX**: `ADX = Smoothed average of DX (usually 14 periods)`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ccxt
from datetime import datetime, timedelta

# Import utilities
from utils.data_downloader import DataDownloader
from utils.indicators import IndicatorCalculator
from utils.backtest_engine import BacktestEngine, StrategyGenerator

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Libraries imported successfully!")

## 3. Data Download

Let's download historical crypto data for our analysis.

In [ ]:
# Initialize data downloader
downloader = DataDownloader()

# Download BTC/USDT data
pair = "BTC/USDT"
timeframe = "1d"
days = 365

print(f"Downloading {pair} data for {days} days...")
df = downloader.get_market_data(pair, timeframe, days, source='ccxt')

if df is not None:
    print(f"\nDownloaded {len(df)} candles of {pair} data")
    print(f"Date range: {df.index[0]} to {df.index[-1]}")
    print(f"\nFirst few rows:")
    display(df.head())
else:
    print("Failed to download data")

## 4. Manual Calculation of ADX

Let's implement ADX from scratch to understand how it works.

In [ ]:
def calculate_adx_manual(high, low, close, period=14):
    """
    Calculate ADX manually
    """
    df = pd.DataFrame({'high': high, 'low': low, 'close': close})
    df['tr1'] = df['high'] - df['low']
    df['tr2'] = abs(df['high'] - df['close'].shift())
    df['tr3'] = abs(df['low'] - df['close'].shift())
    df['tr'] = df[['tr1', 'tr2', 'tr3']].max(axis=1)
    
    df['up_move'] = df['high'] - df['high'].shift()
    df['down_move'] = df['low'].shift() - df['low']
    
    df['plus_dm'] = np.where((df['up_move'] > df['down_move']) & (df['up_move'] > 0), df['up_move'], 0)
    df['minus_dm'] = np.where((df['down_move'] > df['up_move']) & (df['down_move'] > 0), df['down_move'], 0)
    
    df['atr'] = df['tr'].ewm(com=period-1, adjust=False).mean()
    df['plus_di'] = 100 * (df['plus_dm'].ewm(com=period-1, adjust=False).mean() / df['atr'])
    df['minus_di'] = 100 * (df['minus_dm'].ewm(com=period-1, adjust=False).mean() / df['atr'])
    
    df['dx'] = 100 * (abs(df['plus_di'] - df['minus_di']) / (df['plus_di'] + df['minus_di']))
    df['adx'] = df['dx'].ewm(com=period-1, adjust=False).mean()
    
    return df['adx'], df['plus_di'], df['minus_di']

# Calculate ADX
df_manual = df.copy()
df_manual['adx'], df_manual['plus_di'], df_manual['minus_di'] = calculate_adx_manual(df_manual['high'], df_manual['low'], df_manual['close'])

print("ADX calculated manually:")
display(df_manual[['close', 'adx', 'plus_di', 'minus_di']].tail(10))

## 5. Library Implementation

Now let's use established libraries to calculate ADX.

In [ ]:
# Using our utilities
df_utils = df.copy()
df_utils['adx_utils'] = IndicatorCalculator.adx(df_utils['high'], df_utils['low'], df_utils['close'])

# Using pandas-ta
import pandas_ta as ta
df_ta = df.copy()
df_ta.ta.adx(append=True)

# Using TA-Lib
try:
    import talib
    df_talib = df.copy()
    df_talib['adx_talib'] = talib.ADX(df_talib['high'], df_talib['low'], df_talib['close'], timeperiod=14)
    print("TA-Lib ADX calculated successfully")
except ImportError:
    print("TA-Lib not installed, skipping TA-Lib example")
    df_talib = df.copy()
    df_talib['adx_talib'] = np.nan

# Compare results
print("Comparison of different ADX implementations:")
comparison = pd.DataFrame({
    'Manual': df_manual['adx'],
    'Utils': df_utils['adx_utils'],
    'Pandas-TA': df_ta['ADX_14'],
    'TA-Lib': df_talib['adx_talib']
})

display(comparison.dropna().tail(10))

# Check if all methods produce the same results
print("\nDifferences between methods:")
print(f"Manual vs Utils max difference: {abs(comparison['Manual'] - comparison['Utils']).max():.8f}")
print(f"Manual vs Pandas-TA max difference: {abs(comparison['Manual'] - comparison['Pandas-TA']).max():.8f}")

## 6. Visualization

Let's visualize ADX with price action.

In [ ]:
def plot_adx(df, pair):
    """Plot price and ADX"""
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        subplot_titles=(f'{pair} Price', 'ADX'),
                        row_heights=[0.7, 0.3])

    # Candlestick chart
    fig.add_trace(go.Candlestick(x=df.index, 
                                 open=df['open'], 
                                 high=df['high'], 
                                 low=df['low'], 
                                 close=df['close'], 
                                 name='Price'), 
                  row=1, col=1)

    # ADX
    fig.add_trace(go.Scatter(x=df.index, y=df['adx'], mode='lines', name='ADX', line=dict(color='blue', width=2)), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['plus_di'], mode='lines', name='+DI', line=dict(color='green', width=1)), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['minus_di'], mode='lines', name='-DI', line=dict(color='red', width=1)), row=2, col=1)
    fig.add_hline(y=25, line_dash="dash", line_color="gray", row=2, col=1)

    fig.update_layout(title=f'{pair} ADX Analysis',
                      xaxis_title='Date',
                      yaxis_title='Price (USDT)',
                      xaxis_rangeslider_visible=False,
                      height=800)
    fig.update_yaxes(title_text="ADX Value", row=2, col=1)
    fig.show()

plot_adx(df_manual, pair)

## 7. Trading Strategies

### Strategy 1: ADX Trend Strength Filter
- **Concept**: Use ADX to filter trades. Only take long trades when the trend is bullish and ADX is above a certain level (e.g., 25). Only take short trades when the trend is bearish and ADX is above 25.
- **Example**: Combine with a moving average crossover. A buy signal is only valid if ADX > 25. A sell signal is only valid if ADX > 25.
- **Pros**: Helps avoid whipsaws in non-trending markets.
- **Cons**: Can filter out some good trades at the beginning of a trend.

In [ ]:
def adx_filter_strategy(df, adx_threshold=25, short_ma=20, long_ma=50):
    """Generate signals using ADX as a trend filter"""
    df_strat = df.copy()
    df_strat['adx'], _, _ = calculate_adx_manual(df_strat['high'], df_strat['low'], df_strat['close'])
    df_strat['sma_short'] = IndicatorCalculator.sma(df_strat['close'], short_ma)
    df_strat['sma_long'] = IndicatorCalculator.sma(df_strat['close'], long_ma)
    
    # Generate signals
    df_strat['signal'] = 0
    buy_condition = (df_strat['sma_short'] > df_strat['sma_long']) & (df_strat['adx'] > adx_threshold)
    sell_condition = (df_strat['sma_short'] < df_strat['sma_long']) & (df_strat['adx'] > adx_threshold)
    df_strat.loc[buy_condition, 'signal'] = 1
    df_strat.loc[sell_condition, 'signal'] = -1
    df_strat['position'] = df_strat['signal'].diff()
    
    return df_strat

adx_filter_df = adx_filter_strategy(df)

def plot_adx_filter_strategy(df, pair):
    """Plot ADX filter strategy"""
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        subplot_titles=(f'{pair} Price and MAs', 'ADX'),
                        row_heights=[0.7, 0.3])

    # Candlestick chart
    fig.add_trace(go.Candlestick(x=df.index, 
                                 open=df['open'], 
                                 high=df['high'], 
                                 low=df['low'], 
                                 close=df['close'], 
                                 name='Price'), 
                  row=1, col=1)
    
    # SMAs
    fig.add_trace(go.Scatter(x=df.index, y=df['sma_short'], mode='lines', name='SMA(20)', line=dict(color='blue', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['sma_long'], mode='lines', name='SMA(50)', line=dict(color='red', width=1)), row=1, col=1)

    # Buy signals
    buy_signals = df[df['position'] == 1]
    fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['close'], mode='markers', 
                             marker=dict(symbol='triangle-up', color='green', size=10), name='Buy Signal'), row=1, col=1)

    # Sell signals
    sell_signals = df[df['position'] == -2]
    fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['close'], mode='markers', 
                              marker=dict(symbol='triangle-down', color='red', size=10), name='Sell Signal'), row=1, col=1)

    # ADX
    fig.add_trace(go.Scatter(x=df.index, y=df['adx'], mode='lines', name='ADX', line=dict(color='blue', width=2)), row=2, col=1)
    fig.add_hline(y=25, line_dash="dash", line_color="gray", row=2, col=1)

    fig.update_layout(title=f'{pair} ADX Filter Strategy',
                      xaxis_title='Date',
                      yaxis_title='Price (USDT)',
                      xaxis_rangeslider_visible=False,
                      height=800)
    fig.update_yaxes(title_text="ADX Value", row=2, col=1)
    fig.show()

plot_adx_filter_strategy(adx_filter_df, pair)

## 8. Backtesting

Let's backtest the ADX filter strategy to evaluate its performance.

In [ ]:
# Initialize backtest engine
backtester = BacktestEngine(initial_capital=10000, fees=0.001)

# Define strategy using the StrategyGenerator
strategy = StrategyGenerator.adx_filter_strategy(adx_threshold=25, short_ma=20, long_ma=50)

# Run backtest
results = backtester.run(df, strategy)

# Print performance summary
print("ADX Filter Strategy Performance:")
results['performance'].display()

# Plot equity curve
results['trades']['equity_curve'].plot(title='ADX Filter Strategy Equity Curve', figsize=(12, 6))
plt.xlabel('Date')
plt.ylabel('Equity')
plt.show()

## 9. Interpretation and Best Practices

### Interpreting ADX
- **Rising ADX**: The trend is gaining strength.
- **Falling ADX**: The trend is losing strength. This does not mean the trend is reversing, just that it is weakening.
- **+DI and -DI Crossover**: When +DI crosses above -DI, it's a bullish signal. When -DI crosses above +DI, it's a bearish signal.

### Best Practices
- **Don't use ADX alone**: ADX is a complementary indicator. Use it with other indicators to confirm signals.
- **ADX for exiting trades**: A falling ADX can be a signal to take profits as the current trend may be ending.
- **Combine with price patterns**: Use ADX to confirm breakouts from chart patterns.

## 10. Conclusion

The Average Directional Index (ADX) is a powerful tool for measuring trend strength. It helps traders distinguish between trending and non-trending markets, which is crucial for applying the right trading strategy. By using ADX as a filter, traders can improve the reliability of their signals and avoid costly whipsaws.

Remember that no indicator is perfect. Always use ADX as part of a comprehensive trading plan that includes risk management and other forms of analysis.